# isovolume demo with k3d

I am trying to remake the example that I did with ipygany with k3d here.

In [1]:
import numpy as np
import k3d

## Load the data

I can talk in detail later about this but for now, this is just a way to get at a single time step dataset for the ocen depth and o2 concentrations

In [2]:
import intake
from cmip6_preprocessing.preprocessing import combined_preprocessing
# utility func to convert oxygen units
def convert_mol_m3_mymol_kg(o2):
    converted = o2 / 1025 * 1e6
    converted.attrs["units"] = "$\mu mol/kg$"
    return converted


# same cmip6 data as before
col = intake.open_esm_datastore("https://storage.googleapis.com/cmip6/pangeo-cmip6.json")

dd_kwargs = dict(
    zarr_kwargs={'consolidated':True},
    preprocess=combined_preprocessing
)
cat = col.search(variable_id=['deptho'], grid_label='gr', source_id='GFDL-CM4')
ddict = cat.to_dataset_dict(**dd_kwargs)
_, ds = ddict.popitem()
depth = ds.deptho.fillna(0).squeeze().transpose('y','x').load()#.data

cat = col.search(variable_id=['o2'], grid_label='gr', source_id='GFDL-CM4')
ddict = cat.to_dataset_dict(**dd_kwargs)

ds = ddict['CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gr']
da_o2 = convert_mol_m3_mymol_kg(ds.o2).squeeze().isel(time=slice(0,600)).mean('time').transpose('x','y','lev')
# For now I have loaded the coarser depth data, need to find a way to harmonize this with the cooler high res data later


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


The first problem I encountered is that there was no way to implement non-uniform spacing (at least ipygany seems to treat array indicies as units)...

Brute force way to handle this is to interpolate the depth onto regular intervals:

In [3]:
aspect_ratio = {'x':1,'y':1,'lev':1/5}
spacing = {'x':1, 'y':1, 'lev':20}

In [4]:
# the current marching_cube implementation needs regularly spaced data
da_o2 = da_o2.interp(lev=np.arange(0,np.max(depth),spacing['lev']))
o2 = da_o2.fillna(1e12)#.load().data

In [5]:
# transpose both arrays in the same way
depth = depth.transpose('y','x')
o2 = o2.transpose('lev','y','x')

## Plot the bathymetry

In [6]:
plot = k3d.plot()
# xmin, xmax = depth.x.min().data,depth.x.max().data
# ymin, ymax = depth.y.min().data,depth.y.max().data
xmin, xmax = -1.0 ,1.0
ymin, ymax = -1.0 ,1.0
vertical_scaling = - depth.max().data*2

plt_surface = k3d.surface(
    depth.data/vertical_scaling,
    color=5,
    xmin=xmin,
    xmax=xmax,
    ymin=ymin,
    ymax=ymax
)
plot += plt_surface

plot.display()
plot.lighting = 20

Output()

## Now lets add an isosurface

In [7]:
plot2 = k3d.plot()
zmin,zmax = o2.lev.min().data, o2.lev.max().data
iso = k3d.marching_cubes(o2.data, level=80.0,xmin=xmin,
    xmax=xmax,
    ymin=ymin,
    ymax=ymax,
    zmin=zmin/vertical_scaling,
    zmax=zmax/vertical_scaling
    )
plot2 += iso
plot2.display()

/Users/juliusbusecke/miniconda/envs/hack_3d_viz/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


Output()

## And now both together...

In [8]:
plot_combo = k3d.plot()
plot_combo += plt_surface
plot_combo += iso

plot_combo.display()

Output()

In [11]:
# tune the plot
plot_combo.lighting = 50
iso.opacity = 0.5

## Lets get fancy

In [14]:
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

In [15]:
# interactive sliders
plot_combo = k3d.plot()
plot_combo += plt_surface
plot_combo += iso

@interact(o2=widgets.FloatSlider(value=80,min=0,max=400,step=5))
def f(o2):
    iso.level=o2
    
@interact(alpha=widgets.FloatSlider(value=0.8,min=0,max=1,step=0.1))
def g(alpha):
    iso.opacity=alpha

plot_combo.display()

interactive(children=(FloatSlider(value=80.0, description='o2', max=400.0, step=5.0), Output()), _dom_classes=…

interactive(children=(FloatSlider(value=0.8, description='alpha', max=1.0), Output()), _dom_classes=('widget-i…

Output()

## open questions for the hack

- I do not understand the color coding. 
- Can we use xarray to make the aspect ratio/scaling easier?
- I would love to smooth the resulting meshes. Pyvista can do that? 
- Other shading options available?
- map on a sphere?


In [ ]:
# tune the plot

# more light
plot_combo.lighting = 40
iso.opacity = 0.5

# add a slider for the iso level
@interact(x=widgets.FloatSlider(value=0,min=-3,max=4,step=0.01))
def g(x):
    iso.level=x